---
# Agente de IA da Alura
## O Problema
> A empresa de educação Alura deseja criar um chatbot inteligente para auxiliar futuros alunos. Este assistente deve responder a perguntas específicas sobre o conteúdo do curso de RAG, utilizando os próprios PDFs das aulas como sua base de conhecimento. O objetivo é fornecer respostas precisas e confiáveis, baseadas exclusivamente no material didático.

## A Solução
> Construir um sistema RAG completo, seguindo os passos e utilizando as ferramentas discutidas nas aulas

### Etapas
- **Configuração do ambiente**: Instalação das bibliotecas e configuração do modelo local
- **Pipeline de ingestão de dados (ETL)**
  - *Extração*: Carregar os docs PDF das aulas
  - *Transformação*: Aplicar estratégias de chunking adaptativo e gerar embeddings
  - *Carregamento*: Indexar os chunks e seus embeddings em um banco de dados vetorial, o Chroma
- **Construção de um Sistema de Recuperação Avançado**: Implementar busca híbrida para combinar busca lexical *(BM25)* e semântica
- **Criação de uma Cadeia de Conversação Robsuta**
  - Implementar a *ConversationalRetrievalChain*
  - Integrar gerenciamento de memória para manter o contexto do diálogo
  - Aplicar transformação de consulta *(Query Transformation)*, que ocorre internamente na cadeia para refinar as perguntas com base no histórico
- **Avaliação do Sistema com RAGAS**: Avaliar a performance do sistema utilizando as métricas específicas do RAGAS, como *Faithfulness*, *Answer Relevancy*, *Context Precision* e *Context Recall*.


In [ ]:
# Dependências

# pip install langchain-classic langchain-core langchain-community 

# pip install langchain-text-splitters transformers

# pip install langchain-openai langchain-huggingface

# pip install chromadb pypdf rank_bm25

# pip install ragas datasets

### Pipeline de Ingestão de dados

In [2]:
# Extract
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("documentos_curso/")
docs = loader.load()

In [ ]:
# Transform
from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Define que agora o tamanho dos chunks serão em TOKENS, não em CARACTÉRES
def count_tokens(text):
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
    length_function=count_tokens
)


chunks = text_splitter.split_documents(docs)

print(f"Total de chunks criados: {len(chunks)}")

Total de chunks criados: 98


In [ ]:
# Verificar quantidade de TOKENS por chunk
for i, chunk in enumerate(chunks):
    token_count = count_tokens(chunk.page_content)
    print(f"Chunk {i+1}: {token_count} tokens")

Chunk 1: 61 tokens
Chunk 2: 11 tokens
Chunk 3: 50 tokens
Chunk 4: 41 tokens
Chunk 5: 87 tokens
Chunk 6: 91 tokens
Chunk 7: 61 tokens
Chunk 8: 11 tokens
Chunk 9: 35 tokens
Chunk 10: 64 tokens
Chunk 11: 82 tokens
Chunk 12: 93 tokens
Chunk 13: 84 tokens
Chunk 14: 127 tokens
Chunk 15: 109 tokens
Chunk 16: 130 tokens
Chunk 17: 184 tokens
Chunk 18: 126 tokens
Chunk 19: 95 tokens
Chunk 20: 145 tokens
Chunk 21: 201 tokens
Chunk 22: 66 tokens
Chunk 23: 11 tokens
Chunk 24: 9 tokens
Chunk 25: 89 tokens
Chunk 26: 162 tokens
Chunk 27: 74 tokens
Chunk 28: 73 tokens
Chunk 29: 67 tokens
Chunk 30: 133 tokens
Chunk 31: 142 tokens
Chunk 32: 252 tokens
Chunk 33: 125 tokens
Chunk 34: 119 tokens
Chunk 35: 108 tokens
Chunk 36: 333 tokens
Chunk 37: 61 tokens
Chunk 38: 11 tokens
Chunk 39: 11 tokens
Chunk 40: 78 tokens
Chunk 41: 76 tokens
Chunk 42: 33 tokens
Chunk 43: 70 tokens
Chunk 44: 77 tokens
Chunk 45: 71 tokens
Chunk 46: 62 tokens
Chunk 47: 68 tokens
Chunk 48: 65 tokens
Chunk 49: 62 tokens
Chunk 50: 11 to

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

In [4]:
# Load
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embeddings_model
)

print("Banco de dados criado com sucesso!")

Banco de dados criado com sucesso!


### Sistema de Recuperação Avançado

In [ ]:
from langchain_classic.retrievers import BM25Retriever, EnsembleRetriever

# 1. Recuperador Lexical (BM25)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# 2. Recuperador Vetorial (a partir do ChromaDB)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. EnsembleRetriever: combina os resultados
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.4, 0.6] 
)

print("Recuperador de Busca Híbrida Configurado")

Recuperador de Busca Híbrida Configurado


### Cadeia de Conversação Robusta

In [ ]:
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory

# Modelo local
llm = ChatOpenAI(
    model = "google/gemma-2-9b",
    openai_api_base="http://127.0.0.1:1234/v1",
    openai_api_key="lm-studio",
    temperature=0
)

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    output_key = "answer",
    return_messages = True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = ensemble_retriever,
    memory = memory,
    verbose = False,
    return_source_documents = True
)

C:\Users\Juan\AppData\Local\Temp\ipykernel_16480\1398636626.py:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
resposta_teste1 = qa_chain.invoke({"question": "O que é chunking adaptativo?"})
print(resposta_teste1["answer"])	

In [ ]:
resposta_teste2 = qa_chain.invoke({"question": "E quais as principais estratégias?"})
print(resposta_teste2["answer"])

### Avaliação do Sistema com RAGAS

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

# 1. Conjunto de dados para avaliação no RAGAS

perguntas = [
    "O que é RAG e qual problema ele soluciona?",
    "Quais os componentes essenciais do RAG?",
    "Qual a diferença entre busca lexical e semântica?",
    "O que mede a métrica faithfulness do RAGAS?"
]

respostas_puro = [
    "RAG (Retrieval-Augmented Generation) é uma arquitetura que combina um motor de busca para recuperar informações com um L",
    "Os componentes essenciais são: Embeddings, Banco de Dados Vetorial, chunking e um Modelo de Linguagem (LLM).",
    "Busca lexical (como BM25) encontra correspondências exatas de termos, enquanto a busca semântica captura o significado e ",
    "A métrica Faithfulness mede se a resposta gerada é suportada e factualmente consistente com os documentos recuperados, e"
]

C:\Users\Juan\AppData\Local\Temp\ipykernel_16480\4248735931.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
C:\Users\Juan\AppData\Local\Temp\ipykernel_16480\4248735931.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
C:\Users\Juan\AppData\Local\Temp\ipykernel_16480\4248735931.py:3: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
C:\Users\Juan\AppData\Local\Temp\ipykernel_16480\

In [9]:
# 2. Gere as respostas e contextos com a nossa cadeia
respostas_geradas = []
contextos_recuperados = []
for question in perguntas:
    result = qa_chain.invoke({"question": question})
    respostas_geradas.append(result['answer'])
    contextos_recuperados.append([doc.page_content for doc in result['source_documents']])

In [ ]:
# 3. Dataset gerado no formato esperado pelo RAGAS
dataset_dict = {
    'question': perguntas,
    'answer': respostas_geradas,
    'contexts': contextos_recuperados,
    'ground_truth': respostas_puro
}

dataset = Dataset.from_dict(dataset_dict)

In [ ]:
# 4. Execute a avaliação
evaluation_result = evaluate(
    dataset = dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=llm,
    embeddings=embeddings_model
)

# 5. Analise dos resultados
df_resultados = evaluation_result.to_pandas()
print("\nResultados da Avaliação com RAGAS:")
display(df_resultados)

Evaluating:  50%|█████     | 2/4 [01:44<01:43, 51.57s/it]Exception raised in Job[2]: TimeoutError()
Exception raised in Job[0]: TimeoutError()
Evaluating: 100%|██████████| 4/4 [03:00<00:00, 45.01s/it]



Resultados da Avaliação com RAGAS:


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,O que é RAG e qual problema ele soluciona?,[FLUXO COMPLETO DO RAG\nO ciclo de vida comple...,RAG (Retrieval-Augmented Generation) combina a...,RAG (Retrieval-Augmented Generation) é uma arq...,NaN,0.138772,NaN,1.0
